In [ ]:
from __future__ import print_function

import random

import pandas as pd
from tqdm import tqdm
import numpy as np
# from utils.postgres_sql import PostgresSqlConnector

In [ ]:
# a = pd.read_csv('output/ards_data_dynamic/valid_ards_data_with_dynamic_feature_0_to_500.csv')
# base = pd.read_csv('dataset/ards_data/valid_ards_data_with_base_info.csv').loc[:500, :]
#

In [ ]:
from utils.common import combine_csvs

combine_csvs('output/ards_data_dynamic', 'valid_ards_data_with_dynamic_feature_1000_to_5000.csv')
print('end')